In [23]:
import pandas as pd
import collections
import matplotlib.pyplot as plt
import subprocess

In [24]:
condition = str(input('Condition to test: '))

Condition to test: A22.0P5UM


In [25]:
all_phenotypes = pd.read_csv('/home/claudio/Comp_Genetics/results/bugwas_phenotypes.tsv', sep = '\t')
genotypes = pd.read_csv('/home/claudio/Comp_Genetics/raw_data/genotypes_noref_formatted.tsv', sep = '\t')

In [26]:
subset_phenotypes = all_phenotypes[all_phenotypes['pheno'].str.contains(condition)]
genotypes_1 = genotypes.rename(columns={"Unnamed: 0":"ps"})

In [27]:
ini_samples_genotypes = list(genotypes_1.columns.values[2:])
ini_samples_phenotypes = list(subset_phenotypes.iloc[:,0])

# Removing the reference sample from the phenotypes (aka NT12001)
ini_samples_phenotypes = [sample for sample in ini_samples_phenotypes if sample != 'NT12001']

In [40]:
def norm(x):
    y = (x - phenotypes['pheno'].min())/(phenotypes['pheno'].max() - phenotypes['pheno'].min())
    return y

# Which are the samples that do not match
nomatch_samples = [sample for sample in ini_samples_phenotypes if sample not in ini_samples_genotypes]
nomatch_samples = list(dict.fromkeys(nomatch_samples))
    
# Get rid of duplicates and nomatching samples on the phenotypes
phenotypes_1 = subset_phenotypes.drop_duplicates(subset=['ID'])
phenotypes_2 = phenotypes_1[~phenotypes_1['ID'].isin(nomatch_samples) & phenotypes_1['ID'].isin(ini_samples_phenotypes)] ## ~ = negation
phenotypes_2['pheno'] = phenotypes_2['pheno'].map(lambda x: float(x.split('_')[1]).apply(norm)

phenotypes_2.to_csv('/home/claudio/Comp_Genetics/{0}_Results/{0}_Phenotypes.tsv'.format(condition), sep = '\t', index = False)    
# Get sample list to filter
samples_phenotypes = list(phenotypes_2.iloc[:,0])
with open('/home/claudio/Comp_Genetics/{0}_Results/{0}_Filter_Tree.tsv'.format(condition), 'w') as fp:
    fp.write('\n'.join('{}'.format(sample) for sample in samples_phenotypes))
samples_phenotypes.insert(0,'ps')
    
# Subset the genotype dataframe with the phenotype samples
subset_genotypes = genotypes_1[samples_phenotypes]
    
# Remove invariant sites
invariant_sites = []
for i, k in subset_genotypes.iterrows():
    if len(set(k[1:]))== 1:
        invariant_sites.append(False)
    else:
        invariant_sites.append(True)
subset_genotypes = subset_genotypes[invariant_sites]
subset_genotypes.to_csv('/home/claudio/Comp_Genetics/{0}_Results/{0}_Genotypes.tsv'.format(condition), sep = '\t', index = False)

/home/claudio/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [42]:
!bcftools view -S /home/claudio/Comp_Genetics/{condition}_Results/{condition}_Filter_Tree.tsv /home/claudio/Comp_Genetics/input/snps.vcf.gz > /home/claudio/Comp_Genetics/{condition}_Results/{condition}_snps.vcf
!bgzip -c /home/claudio/Comp_Genetics/{condition}_Results/{condition}_snps.vcf > /home/claudio/Comp_Genetics/{condition}_Results/{condition}_snps.vcf.gz
!/home/claudio/Comp_Genetics/tassel-5-standalone/run_pipeline.pl -Xms512m -Xmx10g -importGuess /home/claudio/Comp_Genetics/{condition}_Results/{condition}_snps.vcf.gz -CreateTreePlugin -clusteringMethod Neighbor_Joining -endPlugin -export hello_there -exportType SqrMatrix
!rm /home/claudio/Comp_Genetics/scripts/hello_there1.txt
!mv /home/claudio/Comp_Genetics/scripts/hello_there2.txt /home/claudio/Comp_Genetics/{condition}_Results/{condition}_Distance_Matrix.tsv
!tail -n +6 /home/claudio/Comp_Genetics/{condition}_Results/{condition}_Distance_Matrix.tsv > /home/claudio/Comp_Genetics/{condition}_Results/{condition}_Distance_Matrix_2.tsv
!mv /home/claudio/Comp_Genetics/{condition}_Results/{condition}_Distance_Matrix_2.tsv /home/claudio/Comp_Genetics/{condition}_Results/{condition}_Distance_Matrix.tsv

/home/claudio/Comp_Genetics/tassel-5-standalone/lib/ahocorasick-0.2.4.jar:/home/claudio/Comp_Genetics/tassel-5-standalone/lib/kotlinx-coroutines-core-1.3.0.jar:/home/claudio/Comp_Genetics/tassel-5-standalone/lib/kotlin-stdlib-1.3.50.jar:/home/claudio/Comp_Genetics/tassel-5-standalone/lib/jfreechart-1.0.19.jar:/home/claudio/Comp_Genetics/tassel-5-standalone/lib/slf4j-api-1.7.10.jar:/home/claudio/Comp_Genetics/tassel-5-standalone/lib/biojava-core-4.0.0.jar:/home/claudio/Comp_Genetics/tassel-5-standalone/lib/itextpdf-5.1.0.jar:/home/claudio/Comp_Genetics/tassel-5-standalone/lib/log4j-1.2.13.jar:/home/claudio/Comp_Genetics/tassel-5-standalone/lib/gs-core-1.3.jar:/home/claudio/Comp_Genetics/tassel-5-standalone/lib/snappy-java-1.1.1.6.jar:/home/claudio/Comp_Genetics/tassel-5-standalone/lib/fastutil-8.2.2.jar:/home/claudio/Comp_Genetics/tassel-5-standalone/lib/slf4j-simple-1.7.10.jar:/home/claudio/Comp_Genetics/tassel-5-standalone/lib/scala-library-2.10.1.jar:/home/claudio/Comp_Genetics/tasse

[ForkJoinPool.commonPool-worker-25] INFO net.maizegenetics.pipeline.TasselPipeline - net.maizegenetics.analysis.tree.CreateTreePlugin: time: Sep 17, 2019 17:14:5: progress: 61%
[ForkJoinPool.commonPool-worker-12] INFO net.maizegenetics.pipeline.TasselPipeline - net.maizegenetics.analysis.tree.CreateTreePlugin: time: Sep 17, 2019 17:14:5: progress: 70%
[ForkJoinPool.commonPool-worker-17] INFO net.maizegenetics.pipeline.TasselPipeline - net.maizegenetics.analysis.tree.CreateTreePlugin: time: Sep 17, 2019 17:14:5: progress: 80%
[ForkJoinPool.commonPool-worker-24] INFO net.maizegenetics.pipeline.TasselPipeline - net.maizegenetics.analysis.tree.CreateTreePlugin: time: Sep 17, 2019 17:14:5: progress: 90%
[ForkJoinPool.commonPool-worker-23] INFO net.maizegenetics.pipeline.TasselPipeline - net.maizegenetics.analysis.tree.CreateTreePlugin: time: Sep 17, 2019 17:14:5: progress: 100%
[pool-1-thread-1] INFO net.maizegenetics.analysis.distance.IBSDistanceMatrix3Alleles - IBSDistanceMatrix3Alleles: 

In [14]:
distance_matrix = pd.read_csv('/home/claudio/Comp_Genetics/{0}_Results/{0}_Distance_Matrix.tsv'.format(condition), sep = '\t', header = None)

In [15]:
substitute = list(distance_matrix[0])
substitute.insert(0, 'samples')

In [16]:
distance_matrix.columns = substitute

In [17]:
distance_matrix.to_csv('/home/claudio/Comp_Genetics/{0}_Results/{0}_Distance_Matrix.tsv'.format(condition), sep = '\t', index = None)

In [18]:
subprocess.call (["Rscript", "--vanilla", "/home/claudio/Comp_Genetics/scripts/13_Tree_from_matrix.R", condition])

0

In [41]:
subprocess.call (["nohup","Rscript", "--vanilla", "/home/claudio/Comp_Genetics/scripts/14_General_LinLoc.R", condition, ">", "/home/claudio/Comp_Genetics/{0}_Results/Rout.out".format(condition), "&"])

KeyboardInterrupt: 